In [ ]:
import os

In [ ]:
work_dir = '/content/drive/MyDrive/DREAMS-LAB'

In [ ]:
os.listdir(work_dir+'/Multispectral')

['Krutarth-Multispectral test images', 'Week2', 'Week5', 'Week3']

In [ ]:
from __future__ import print_function

import cv2
import numpy as np 
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
MAX_FEATURES = 2000
GOOD_MATCH_PERCENT = 0.5

def alignImages(im1, im2):

  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)

  # Sort matches by score
  # print(type(matches))
  # matches.sort(key=lambda x: x.distance, reverse=False)
  matches = sorted(matches, key = lambda x:x.distance)


  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]

  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)

  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)

  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))

  return im1Reg, h

In [ ]:
def create_NDVI_map(img):

  print(img)

  img1 = cv2.imread(img+'_1.tif')
  img2 = cv2.imread(img+'_2.tif')
  img3 = cv2.imread(img+'_3.tif')
  img4 = cv2.imread(img+'_4.tif')
  img5 = cv2.imread(img+'_5.tif')

  print(img3.shape)

  print("Aligning images ...")
  # Registered image will be resotred in imReg.
  # The estimated homography will be stored in h.
  imReg, h = alignImages(img3, img5)

  # Write aligned image to disk.
  outFilename = "aligned.jpg"
  print("Saving aligned image : ", outFilename);
  cv2.imwrite(outFilename, imReg)

  # Print estimated homography
  # print("Estimated homography : \n",  h)

  img_aligned = cv2.imread("aligned.jpg")
  RED = np.array(img_aligned[:,:,0], dtype = np.float64)
  NIR = np.array(img5[:,:,0], dtype = np.float64)

  NDVI = (NIR-RED)/(NIR+RED)

  plt.figure(figsize=(20,10))
  plt.imshow(NDVI, cmap='hot')
  plt.colorbar()
  plt.savefig(img+'_NDVI.png')
  plt.close() 

  # Creating RGB image 

  im_b, h = alignImages(img1, img2)
  outFilename = "aligned_blue.jpg"
  cv2.imwrite(outFilename, im_b)

  im_blue = cv2.imread("aligned_blue.jpg")
  im_r, h = alignImages(img3, img2)
  outFilename = "aligned_red.jpg"
  cv2.imwrite(outFilename, im_r)

  t = (im_r[:,:,0].reshape(960, 1280, 1), img2[:,:,0].reshape(960, 1280, 1), im_b[:,:,0].reshape(960, 1280, 1))
  rgb_img = np.concatenate(t, axis=-1)
  rgb_img = np.array(rgb_img)

  # plt.imsave(img+"_rgb.png", rgb_img)
  plt.imsave("_rgb.png", rgb_img)



In [ ]:
for week_dir in os.listdir(work_dir+"/Multispectral"):

  if "Week" not in week_dir:
    continue 
  
  for sub_week_dir in os.listdir(work_dir+'/Multispectral'+'/'+week_dir):

    temp = os.listdir(work_dir+'/Multispectral'+'/'+week_dir+'/'+sub_week_dir)
    l = [x for x in temp if "IMG" in x]
    l.sort()

    imgs = []
    itr = 0
    
    while(itr < len(l)):
      img = l[itr].split('_')
      imgs.append(work_dir+'/Multispectral'+'/'+week_dir+'/'+sub_week_dir+'/'+img[0]+'_'+img[1])
      itr += 5


    for img in imgs:
      create_NDVI_map(img)
    
  #   break
  # break

/content/drive/MyDrive/DREAMS-LAB/Multispectral/Week2/Week2_Apr28_11AM/IMG_0000
(960, 1280, 3)
Aligning images ...
Saving aligned image :  aligned.jpg
/content/drive/MyDrive/DREAMS-LAB/Multispectral/Week2/Week2_Apr28_11AM/IMG_0001
(960, 1280, 3)
Aligning images ...
Saving aligned image :  aligned.jpg
/content/drive/MyDrive/DREAMS-LAB/Multispectral/Week2/Week2_Apr28_11AM/IMG_0002
(960, 1280, 3)
Aligning images ...
Saving aligned image :  aligned.jpg
/content/drive/MyDrive/DREAMS-LAB/Multispectral/Week2/Week2_Apr28_11AM/IMG_0003
(960, 1280, 3)
Aligning images ...
Saving aligned image :  aligned.jpg
/content/drive/MyDrive/DREAMS-LAB/Multispectral/Week2/Week2_Apr28_11AM/IMG_0004
(960, 1280, 3)
Aligning images ...
Saving aligned image :  aligned.jpg
/content/drive/MyDrive/DREAMS-LAB/Multispectral/Week2/Week2_Apr28_11AM/IMG_0005
(960, 1280, 3)
Aligning images ...
Saving aligned image :  aligned.jpg
/content/drive/MyDrive/DREAMS-LAB/Multispectral/Week2/Week2_Apr28_11AM/IMG_0006
(960, 1280, 3)